In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.manifold import MDS
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR

c:\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_x = pd.read_csv(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\data\train_x.csv", index_col=0)

In [3]:
train_y = pd.read_csv(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\data\train_y.csv", index_col=0)

In [ ]:
train_y

,ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage
2023-03-16 01:00:00+00:00,41.5
2023-03-16 01:00:01+00:00,41.5
2023-03-16 01:00:02+00:00,41.5
2023-03-16 01:00:03+00:00,41.5
2023-03-16 01:00:04+00:00,41.5
...,...
2023-04-11 19:59:56+00:00,41.5
2023-04-11 19:59:57+00:00,41.5
2023-04-11 19:59:58+00:00,41.5
2023-04-11 19:59:59+00:00,41.5


In [ ]:
def add_time_features(df):
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    daytime_arr = []
    for idx in df.index:
        if df.loc[idx, "hour"] >= 6 and df.loc[idx, "hour"] <= 18:
            daytime_arr.append(1)
        else:
            daytime_arr.append(0)

    df['day_time'] = daytime_arr
    df['night_time'] = 1 - df['day_time']
    return df

In [ ]:
train_x.index = pd.to_datetime(train_x.index)
train_y.index = pd.to_datetime(train_y.index)

In [ ]:
train_x.day_time

2023-03-16 01:00:00+00:00    0
2023-03-16 01:00:01+00:00    0
2023-03-16 01:00:02+00:00    0
2023-03-16 01:00:03+00:00    0
2023-03-16 01:00:04+00:00    0
                            ..
2023-04-11 19:59:56+00:00    0
2023-04-11 19:59:57+00:00    0
2023-04-11 19:59:58+00:00    0
2023-04-11 19:59:59+00:00    0
2023-04-11 20:00:00+00:00    0
Name: day_time, Length: 940513, dtype: int64

In [ ]:
train_x = add_time_features(train_x)
train_y = add_time_features(train_y)

In [ ]:
train_x_day = train_x[train_x.day_time == 1]

In [ ]:
train_y_day = train_x[train_x.day_time == 1]

In [ ]:
train_x_day = train_x_day.drop(columns=["day_time", "night_time"])
train_y_day = train_y_day.drop(columns=["day_time", "night_time"])

In [ ]:
scaler = StandardScaler()
standardized_train = scaler.fit_transform(train_y_day)
scaled_df = pd.DataFrame(standardized_train, columns=train_x.columns)
scaled_df = scaled_df.drop(columns=["hour", "minute"])

In [ ]:
pca = PCA(n_components=len(scaled_df.columns))  # You can specify the number of components you want
X_pca = pca.fit_transform(scaled_df)


In [ ]:
model = SVR(kernel="rbf")

In [ ]:
model.fit(train_x_day[:500000], train_y_day.iloc[:,0][:500000])

SVR()

In [ ]:
y_pred = model.predict(train_x_day[500000:510000])

In [ ]:
mean_absolute_error(train_y_day[500000:510000], y_pred)

0.09512403624058288